# Installs

In [1]:
# !pip -q install pydantic==2.5.2 
# !pip -q install langchain langchain_mistralai langchain-openai
# !pip -q install sqlalchemy
# !pip -q install huggingface_hub openai tiktoken
# !pip -q install chromadb duckduckgo-search

## Logging

## Imports

In [2]:
import llmtools as t

In [3]:
t.set_openai_key()
logger = t.setup_logging()

2024-02-19 17:20:35,540 [MainThread  ] [INFO ]  ---Setup logging---
INFO:rootLogger:---Setup logging---


In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.llms import OpenAI

from langchain.schema.output_parser import StrOutputParser


In [5]:
model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
    )

model2 = OpenAI(
    model="text-davinci-003",
    temperature=0
    )

In [6]:
prompt = ChatPromptTemplate.from_template(
    "tell me an intersting fact about {subject}"
    )

In [7]:
chain = prompt | model
chain2 = prompt | model2

In [11]:
chain.invoke({"subject": "langchain"})

AIMessage(content='One interesting fact about Langchain is that it is a decentralized platform that aims to revolutionize the way language learning is conducted by utilizing blockchain technology. This allows for transparent and secure transactions between students and teachers, as well as the ability to earn rewards for completing language learning tasks.')

In [12]:
chain = prompt | model | StrOutputParser()

In [14]:
chain.invoke({"subject": "Zimbabwe"})

'Zimbabwe is home to the ancient city of Great Zimbabwe, which was once the capital of the Kingdom of Zimbabwe and is now a UNESCO World Heritage Site. The city was built entirely of stone without the use of mortar, and its ruins are a testament to the advanced engineering and architectural skills of the people who lived there.'

# Bindings

In [16]:
prompt = ChatPromptTemplate.from_template(
    "tell me 3 intersting facts about {subject}"
    )

In [17]:
chain = prompt | model.bind(stop=["\n"]) | StrOutputParser()
chain.invoke({"subject": "go language"})

'1. Go was developed by Google in 2007 and was officially released to the public in 2009. It was created by Robert Griesemer, Rob Pike, and Ken Thompson, who are all well-known computer scientists.'

# OpenAI Functions

In [19]:
functions = [
    {
      "name": "joke",
      "description": "A joke",
      "parameters": {
        "type": "object",
        "properties": {
          "setup": {
            "type": "string",
            "description": "The setup for the joke"
          },
          "punchline": {
            "type": "string",
            "description": "The punchline for the joke"
          }
        },
        "required": ["setup", "punchline"]
      }
    }
  ]
functions_chain = prompt | model.bind(function_call= {"name": "joke"}, functions= functions)

In [20]:
functions_chain.invoke({'subject': 'football'}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"setup":"Why was the football coach angry when he went to the bank?","punchline":"He wanted his quarterback!"}', 'name': 'joke'}})

# Functions Ouptut Parser

In [25]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

functions_chain = (
    prompt
    | model.bind(function_call= {"name": "joke"}, functions= functions)
    | JsonKeyOutputFunctionsParser(key_name='setup')
)

In [26]:
functions_chain.invoke({"subject": "bears"})

"Why don't bears like fast food?"

# Retriever

In [27]:
from operator import itemgetter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough

In [28]:
# Create the retriever
fake_docs = ["James bond works for MI6","Bond is a spy",
             "James Bond has a licence to kill", "James Bond likes cats"]
vectorstore = Chroma.from_texts(fake_docs, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

/Users/tapiwamaruni/anaconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [29]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [30]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [31]:
chain.invoke("Who is James Bond?")

'James Bond is a spy who works for MI6 and has a licence to kill. He also likes cats.'

In [32]:
chain.invoke("What does James Bond like to do?")

'James Bond likes cats.'

In [33]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = {
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question"),
    "language": itemgetter("language")
} | prompt | model | StrOutputParser()

In [34]:
chain.invoke({"question": "where does James work?", "language": "english"})

'James Bond works for MI6.'

In [35]:
chain.invoke({"question": "where does James work?", "language": "shona"})

'James Bond anoratidza kubva kuMI6.'

# Tools

In [36]:
from langchain.tools import DuckDuckGoSearchRun

In [37]:
search = DuckDuckGoSearchRun()

In [38]:
template = """turn the following user input into a search query for a search engine:

{input}"""

prompt = ChatPromptTemplate.from_template(template)

In [39]:
chain = prompt | model | StrOutputParser() | search

In [42]:
chain.invoke({"input": "Who played james bond first"})

'"actor who played James Bond first"'

In [41]:
chain = prompt | model | StrOutputParser()
chain.invoke({"input": "Who played james bond last"})

'"actor who played James Bond last"'

In [43]:
from langchain.schema.runnable import RunnableLambda

def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain1 = prompt | model

chain = {
    "a": itemgetter("foo") | RunnableLambda(length_function),
    "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")} | RunnableLambda(multiple_length_function)
} | prompt | model

In [44]:
chain.invoke({"foo": "bars", "bar": "gahs"})

AIMessage(content='4 + 16 equals 20.')